In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
print('Ready ✓')

In [ ]:
def try_parse_date(s: str, formats=None):
    """يحاول قراءة التاريخ بعدة صيغ ويرجع NaT إذا فشل."""
    if not isinstance(s, str):
        return pd.NaT
    if formats is None:
        formats = ("%Y-%m-%d", "%Y/%m/%d", "%d-%m-%Y")
    for fmt in formats:
        try:
            return datetime.strptime(s, fmt).date()
        except Exception:
            pass
    return pd.NaT

In [ ]:
def clean_transform(df: pd.DataFrame, 
                    text_cols=None,
                    date_cols=None,
                    num_cols=None,
                    add_features=False) -> pd.DataFrame:
    """
    تنظيف وتحويل البيانات بشكل عام.
    - text_cols: أعمدة نصية يتم تنظيفها.
    - date_cols: أعمدة تواريخ يتم تحويلها.
    - num_cols: أعمدة رقمية يتم تحويلها لأرقام.
    - add_features: لو True يضيف أعمدة مشتقة (مثل year/month).
    """
    df = df.copy()

    # تنظيف النصوص
    if text_cols:
        for col in text_cols:
            df[col] = df[col].astype(str).str.strip().str.replace(r"\s+", " ", regex=True)

    # تحويل أرقام
    if num_cols:
        for col in num_cols:
            df[col] = (
                df[col].astype(str)
                .str.replace(r"[^\d\.\,]", "", regex=True)
                .str.replace(",", "", regex=False)
                .replace({"": np.nan})
            )
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # تحويل تواريخ
    if date_cols:
        for col in date_cols:
            df[col] = df[col].apply(try_parse_date)

    # إزالة الصفوف الناقصة
    df = df.dropna()

    # أعمدة مشتقة اختيارية
    if add_features and date_cols:
        for col in date_cols:
            df[f"{col}_year"] = pd.to_datetime(df[col]).dt.year
            df[f"{col}_month"] = pd.to_datetime(df[col]).dt.month

    return df

In [ ]:
def summarize(df: pd.DataFrame) -> pd.DataFrame:
    return pd.DataFrame({
        "rows": [len(df)],
        "first_date": [str(pd.to_datetime(df[col]).min().date()) if len(df) else None for col in df.select_dtypes(include=['datetime64']).columns][0] if any(df.select_dtypes(include=['datetime64']).columns) else None,
        "last_date": [str(pd.to_datetime(df[col]).max().date()) if len(df) else None for col in df.select_dtypes(include=['datetime64']).columns][0] if any(df.select_dtypes(include=['datetime64']).columns) else None,
    })

In [ ]:
# مثال تطبيقي على نفس بيانات المبيعات
raw_rows = [
    [1,  "2025-08-31", " Phone  X  ", "2,499.00",  "1",   "Riyadh", "C001"],
    [2,  "2025/08/31", "Headphones",  "399.99",    "2",   "Jeddah", "C002"],
    [3,  "31-08-2025", "Charger",     "89.5 SAR",  "3",   "Taif",   "C003"]
]
raw_df = pd.DataFrame(raw_rows, columns=["order_id","date","product","price","quantity","city","customer_id"])
print("🔹 Raw Data")
display(raw_df.head())

clean_df = clean_transform(raw_df, text_cols=["product","city","customer_id"], date_cols=["date"], num_cols=["price","quantity"], add_features=True)
print("🔹 Cleaned Data")
display(clean_df.head())
print("🔹 Summary")
display(summarize(clean_df))

In [ ]:
# تخزين في SQLite
conn = sqlite3.connect('data_clean.db')
clean_df.to_sql('clean_table', conn, if_exists='replace', index=False)
print("✅ تم تخزين البيانات في قاعدة SQLite باسم data_clean.db ✓")
conn.close()